In [2]:
import numpy as np

In [16]:
def calculate_probability(data, name, probs = {}):
    data_for_name = data[name]
    dependencies = data_for_name[0]
    print('dependencies', dependencies)
    dependencies_probs = []
    for dep in dependencies:
        if dep not in probs:
            calculate_probability(data, dep, probs)
        dependencies_probs.add(probs[dep])
    all_length = np.multiply([len(dep) for dep in dependencies_probs])
    accumulated = np.ones(all_length)
    repeats = all_length
    tails = 1
    for dep in dependencies_probs:
        accumulated = accumulated * np.

SyntaxError: invalid syntax (<ipython-input-16-f933349689f9>, line 14)

In [12]:
animals= \
    {
        'Animal':[
            (),
            [None, ['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle']],
            np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        ],
        'Environment':[
            ('Animal'),
            [['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle'], ['Air', 'Land', 'Water']],
            np.array(
                [
                    [0, 1, 0],
                    [0, 0.5, 0.5],
                    [0, 0, 1],
                    [0.5, 0.5, 0],
                    [0, 0.5, 0.5]
                ]
            )
        ],
        'HasShell':
        [
            ('Animal'),
            [['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle'], ['True', 'False']],
            np.array(
                [
                    [0, 1],
                    [0, 1],
                    [0, 1],
                    [0, 1],
                    [1, 0]
                ]
            )
        ],
        'BearsYoungAs':
        [
            ('Animal'),
            [['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle'], ['Live', 'Eggs']],
            np.array(
                [
                    [1, 0],
                    [0, 1],
                    [0, 1],
                    [0, 1],
                    [0, 1]
                ]
            )
        ],
        'Class':
        [
            ('Animal'),
            [['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle'], ['Bird', 'Mammal', 'Reptile']],
            np.array(
                [
                    [0, 1, 0],
                    [1, 0, 0],
                    [0, 1, 0],
                    [1, 0, 0],
                    [0, 0, 1],
                ]
            )
        ],
        'WarmBlooded':
        [
            ('Class'),
            [['Bird', 'Mammal', 'Reptile'], ['True', 'False']],
            np.array(
                [
                    [1, 0],
                    [1, 0],
                    [0, 1]
                ]
            )
        ],
            'BodyCovering':
        [
            ('Class'),
            [['Bird', 'Mammal', 'Reptile'], ['Fur', 'Feathers', 'Scales']],
            np.array(
                [
                    [0, 1, 0],
                    [1, 0, 0],
                    [0, 0, 1]
                ]
            )
        ]
    }

In [4]:
x1 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [5]:
x2 = np.array(
                [
                    [0, 1, 0],
                    [0, 0.5, 0.5],
                    [0, 0, 1],
                    [0.5, 0.5, 0],
                    [0, 0.5, 0.5]
                ]
            )

In [9]:
x1*x2.T

array([[ 0. ,  0. ,  0. ,  0.1,  0. ],
       [ 0.2,  0.1,  0. ,  0.1,  0.1],
       [ 0. ,  0.1,  0.2,  0. ,  0.1]])